In [2]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
from midi.utils import midiread, midiwrite
import sys
sys.path.append("/Users/danshiebler/Documents")
import helper_functions as hf
# sys.path.append('/home/hanhong/Projects/python27/DeepLearningTutorials/code/')
# from utils import tile_raster_images
# import Image
from tensorflow.python.ops import control_flow_ops
from tqdm import tqdm
from matplotlib import pyplot as plt
import glob
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels,\
    mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [32]:

def build_rbm(x, W, bv, bh, k):
    size_bt = tf.shape(x)[0]

    # helper functions
    def sample(probs):
        return tf.to_float(tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1)))

    def sampleInt(probs):
        return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))

    # define graph/algorithm

    # CD-k
    # we use tf.while_loop to achieve the multiple (k - 1) gibbs sampling  
    # set up tf.while_loop()
    def rbmGibbs(xx, hh, count, k):
        xk = sampleInt(tf.sigmoid(tf.matmul(hh, tf.transpose(W)) + tf.tile(tf.transpose(bv), [size_bt, 1])))
        hk = sampleInt(tf.sigmoid(tf.matmul(xk, W) + tf.tile(tf.transpose(bh), [size_bt, 1])))
        return xk, hk, count+1, k

    def lessThanK(xk, hk, count, k):
        return count <= k

    ct = tf.constant(1) #counter
    h = sample(tf.sigmoid(tf.matmul(x, W) + tf.tile(bh, [size_bt, 1])))# sample h x1 h1 ..
    [xk1, hk1, _, _] = control_flow_ops.While(lessThanK, rbmGibbs, [x, h, ct, tf.constant(k)], 1, False)

    def free_energy(xx):
        #return -(v * bv).sum() - T.log(1 + T.exp(T.dot(v, W) + bh)).sum()
        A = -tf.sigmoid(tf.matmul(xx, W) + tf.tile(bh, [size_bt, 1]))
        B = -tf.reduce_sum(tf.matmul(xx, tf.transpose(tf.cast(tf.tile(bv, [size_bt, 1]), tf.float32))), 1)
        return tf.add(B, A)

    #Define loss and optimizer
    #the cost is based on the difference in free energy between v and v_sample
    e1 = free_energy(x)
    e2 = free_energy(xk1)
    cost = tf.reduce_mean(tf.sub(e1, e2))
    return xk1, cost


In [33]:
def build_rnnrbm(n_visible= 88, n_hidden= 150, n_hidden_recurrent= 100):


    # variables and place holder
    W  = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01))
    bh = tf.Variable(tf.random_uniform([n_hidden], -0.005, 0.005))
    bv = tf.Variable(tf.random_uniform([n_visible], -0.005, 0.005))
    x  = tf.placeholder(tf.float32, [None, n_visible])
    a  = tf.placeholder(tf.float32)
    size_bt = tf.shape(x)[0]

    Wuh = tf.Variable(tf.random_normal([n_hidden_recurrent, n_hidden], 0.01))
    Wuv = tf.Variable(tf.random_normal([n_hidden_recurrent, n_visible], 0.01))
    Wvu = tf.Variable(tf.random_normal([n_visible, n_hidden_recurrent], 0.01))
    Wuu = tf.Variable(tf.random_normal([n_hidden_recurrent, n_hidden_recurrent], 0.01))
    bu = tf.Variable(tf.random_uniform([n_hidden_recurrent, 1], -0.005, 0.005))

    params = W, bh, bv, x, a, Wuh, Wuv, Wvu, Wuu, bu

    # If `v_t` is given, deterministic recurrence to compute the variable
    # biases bv_t, bh_t at each time step. If `v_t` is None, same recurrence
    # but with a separate Gibbs chain at each time step to sample (generate)
    # from the RNN-RBM. The resulting sample v_t is returned in order to be
    # passed down to the sequence history.
    def recurrence(xx, u_tm1, _0, _1, count, k):
        bv_t = tf.add(bv, tf.matmul(u_tm1, Wuv))
        bh_t = tf.add(bh, tf.matmul(u_tm1, Wuh))
        sl = tf.slice(xx, [count, -1], [1, n_visible])
        u_t  = tf.tanh(bu + tf.matmul(sl, Wvu) + tf.matmul(u_tm1, Wuu))
        return [xx, u_tm1, bv_t, bh_t, count+1, k]

    def lessThanNumIter(_0, _1, _2, _3, count, num_iter):
        return count <= num_iter

    ct = tf.constant(1, tf.int32) #counter
    u0 = tf.zeros([1, n_hidden_recurrent], tf.float32)
    bh_t = tf.random_uniform([1, n_hidden], -0.005, 0.005)
    bv_t = tf.random_uniform([1, n_visible], -0.005, 0.005)
    [x, u_t, bv_t, bh_t, _, _] = control_flow_ops.While(lessThanNumIter, recurrence, [x, u0, bv_t, bh_t, ct, size_bt])

    #Build this rbm based on the bias vectors that we already found 
    sample, cost = build_rbm(x, W, bv_t, bh_t, k=15)
    return x, sample, cost, params

x, sample, cost, params = build_rnnrbm()
W, bh, bv, x, a, Wuh, Wuv, Wvu, Wuu, bu = params
optimizer = tf.train.AdamOptimizer(learning_rate=a).minimize(cost)


In [34]:
#FILL THIS IN

batch_size = 100 # batch size

def get_songs(path='data/music_all/train/*.mid'):
    files = glob.glob(path)
    f = files[0]
    r=(21, 109)
    dt=0.3
    songs = [midiread(f, r, dt).piano_roll for f in tqdm(files)]
    return songs

# songs = get_songs()
with tf.Session():
    init = tf.initialize_all_variables()
    sess.run(init)

    # loop with batch
    for song in songs:
        for i in range(1, len(song), batch_size):
            tr_x = song[i:i + batch_size]
            alpha = min(0.05, 100/float(i))
            sess.run(optimizer, feed_dict={x: tr_x, a: alpha})
   


InvalidArgumentError: Incompatible shapes: [100] vs. [100,150]
	 [[Node: gradients_5/Add_4_grad/BroadcastGradientArgs = BroadcastGradientArgs[_device="/job:localhost/replica:0/task:0/cpu:0"](gradients_5/Add_4_grad/Shape, gradients_5/Add_4_grad/Shape_1)]]
Caused by op u'gradients_5/Add_4_grad/BroadcastGradientArgs', defined at:
  File "/Users/danshiebler/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/danshiebler/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/danshiebler/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/danshiebler/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/danshiebler/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 403, in start
    ioloop.IOLoop.instance().start(

In [7]:
sess.close()
